In [1]:
# The C2QA package is currently not published to PyPI.
# To use the package locally, add the C2QA repository's root folder to the path prior to importing c2qa.
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Cheat to get MS Visual Studio Code Jupyter server to recognize Python venv
module_path = os.path.abspath(os.path.join("../../venv/Lib/site-packages"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from quspin.operators import hamiltonian  # operators
from quspin.basis import boson_basis_1d  # Hilbert space boson basis
from quspin.basis import tensor_basis, spinless_fermion_basis_1d  # Hilbert spaces
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function, division
from quspin.tools.Floquet import Floquet_t_vec
from quspin.tools.evolution import evolve  # ODE evolve tool
from quspin.operators import hamiltonian  # operators
from quspin.basis import boson_basis_1d  # Hilbert space boson basis
from quspin.basis import tensor_basis, spinless_fermion_basis_1d  # Hilbert spaces
from quspin.basis import spin_basis_1d  # Hilbert space spin basis
import copy
import c2qa
import util
from scipy.sparse.linalg import eigsh

In [3]:
util.energy_scaling()

E[0]  -4.000000000000006  E[1]  -4.000000000000009  delta  2.6645352591003757e-15


### Build basis and Hamiltonian

In [3]:
Nsites = 4
Nbosons = 2
###### parameters
L_spin = Nsites
L_modes = Nsites  # system size
cutoff = Nbosons + 1  #sites+2
h = 1  # field strength
t = 1

### Build projector onto gauge conserving basis

In [4]:
P_sparse = util.gaussProjector(Nsites, Nbosons)
P_sparse

<20x1296 sparse matrix of type '<class 'numpy.complex128'>'
	with 320 stored elements in Compressed Sparse Row format>

In [5]:
# building the two bases to tensor together
basis_spin = spin_basis_1d(L=L_spin)
basis_boson = boson_basis_1d(L=L_modes, sps=cutoff)
# print(basis_boson)
basis = tensor_basis(basis_spin, basis_boson)
# print(basis)
# print(basis.index("10","010"))

# Ratios of hopping and field

In [74]:
hopping_strength = -0.01
field_strength = 10
Hgaugefixed = util.build_H(hopping_strength, field_strength, L_modes, L_spin, P_sparse, basis, True)
E, psi0 = eigsh(Hgaugefixed, k=11, which='SA')
print(E)

[-40.00012    -40.0001     -40.00008    -40.         -40.00002
 -40.0001     -40.00002    -40.00004    -40.00004    -40.00004
 -20.00005362]


In [ ]:
# States with weight
for i in range(len(psi0)):
    print(np.abs(psi0[i]))

psi0_notgaugefixed = P_sparse.T @ psi0

### Correlators

In [8]:
util.gauge_invariant_correlator(hopping_strength, field_strength, Nsites, psi0_notgaugefixed, Nbosons, basis)

[[ 1.76152093  2.23847907  2.23847907  1.76152093]
 [ 1.78583734  2.0244158   1.78583734 -1.        ]
 [ 1.49678554  1.49678554 -1.         -1.        ]
 [ 1.06337245 -1.         -1.         -1.        ]]


In [9]:
util.pairing_correlator(hopping_strength, field_strength, Nsites, psi0_notgaugefixed, Nbosons, basis)

## Energies

#### Hopping dominates: ground state is (1/2 |2+0> + 1/2 |0+2> + 1/sqrt(2) |1-1>)

#### Positive field dominates so |-> lowers the energy the most: ground state is |1-1>

### $J > \lambda$

In [154]:
# ED Z2 LGT
elist = []
hopping_strength = -1
field_strength = 0
Hgaugefixed = util.build_H(hopping_strength, field_strength, L_modes, L_spin, P_sparse, basis, True)
#E, psi0 = eigsh(Hgaugefixed, k=20, which='SA')
E, psi0 = np.linalg.eigh(Hgaugefixed.todense())
elist.append(E)

# Analytical free bosons
elist_freebosons = []
E_k=-2*hopping_strength*np.cos(2*np.pi*np.arange(0,L_modes,1)/L_modes)
elist_freebosons.append(E_k)

# print(np.sort(E))
# print(np.sort(E_k))
# print(E[1]-E[0])
# print(psi0)
# psi0[:,6]

for i in range(len(elist)):
    plt.plot(range(len(elist[i])),elist[i],".",label="ED Z2LGT no field, 1 boson")
    plt.plot(range(len(elist_freebosons[i])),np.flip(elist_freebosons[i]),"x",label="Analytical, 1 free boson")
# plt.xlabel("ED: Eigenenergy number/ Analytical: Momentum ($2\pi/L$)")
plt.ylabel("Eigenenergy")
plt.xlabel("ED: eigenenergy number/ analytical: momentum ($2\pi/L$)")
plt.title(r"$J \greater \lambda$, "+str(Nsites)+" sites")
plt.legend(loc="lower right")
plt.show()

### $J < \lambda$

In [14]:
hopping_strength = -0.1
field_strength = 1

# ED Z2 LGT
elist = []
Hgaugefixed = util.build_H(hopping_strength, field_strength, L_modes, L_spin, P_sparse, basis, True)
#E, psi0 = eigsh(Hgaugefixed, k=20, which='SA')
E, psi0 = np.linalg.eigh(Hgaugefixed.todense())
elist.append(E)

# Analytical free bosons
elist_freebosons = []
E_k=-2*1*np.cos(2*np.pi*np.arange(0,L_modes,1)/(2*L_modes))
elist_freebosons.append(E_k)

# ED paired bosons
elist_pairedbosons = []
basis_boson2 = boson_basis_1d(L=L_modes, sps=cutoff, Nb=Nbosons)
print(basis_boson2)
hamiltonian_paired_hopping = util.build_H_paired_hopping(-1, L_modes, basis_boson2, periodicBC=True)
print(hamiltonian_paired_hopping)
E, psi0 = np.linalg.eig(hamiltonian_paired_hopping.todense())
print("energy ",E)#,psi0[:,0],E[0])
print(psi0)
elist_pairedbosons.append(E)

for i in range(len(elist)):
    plt.plot(range(len(elist[i])),elist[i],".",label="ED Z2LGT, 2 bosons")
    plt.plot(range(len(elist_pairedbosons[i])),elist_pairedbosons[i],".",label="ED paired bosons, 2 bosons")
    plt.plot(range(len(elist_freebosons[i])),np.flip(elist_freebosons[i]),"x",label="Analytical, 1 free boson")
# plt.xlabel("ED: Eigenenergy number/ Analytical: Momentum ($2\pi/L$)")
plt.ylabel("Eigenenergy")
plt.xlabel("ED: eigenenergy number/ analytical: momentum ($2\pi/L$)")
plt.title(r"$J \less \lambda$, "+str(Nsites)+" sites")
plt.legend()
plt.show()

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |2 0 0 0>           54  
      1.         |1 1 0 0>           36  
      2.         |1 0 1 0>           30  
      3.         |1 0 0 1>           28  
      4.         |0 2 0 0>           18  
      5.         |0 1 1 0>           12  
      6.         |0 1 0 1>           10  
      7.         |0 0 2 0>            6  
      8.         |0 0 1 1>            4  
      9.         |0 0 0 2>            2  
static mat: 
  (0, 4)	(-2+0j)
  (0, 9)	(-2+0j)
  (4, 0)	(-2+0j)
  (4, 7)	(-2+0j)
  (7, 4)	(-2+0j)
  (7, 9)	(-2+0j)
  (9, 0)	(-2+0j)
  (9, 7)	(-2+0j)


dynamic:



LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

# Energy gap

In [162]:
set=[]
lab = []

In [163]:
res = util.energy_gap(set, lab, -1, 1, L_modes, L_spin, P_sparse, basis, Nbosons)
set = res[0]
energy0 = res[1]
energy1 = res[2]
lab = res[3]

done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 
done 


In [164]:
xlabelplot = "Value of the field term (hopping=1)"
plt.title("Z2LGT Energy gaps")
for i in range(len(set)):
    plt.plot(energy0[0],energy0[1],label="E0 "+lab[i])
for i in range(len(set)):
    plt.plot(energy1[0],energy1[1],label="E1 "+lab[i])
plt.xlabel(xlabelplot)
plt.ylabel("Eigenvalue")
plt.legend()
plt.show()

plt.title("Z2LGT Energy gap")
for i in range(len(set)):
    plt.plot(set[i][0],set[i][1],label=lab[i])
    # plt.plot(set[i][0],set[i][0]**(-1),".",label="$x^{-2}$")
plt.xlabel(xlabelplot)
plt.ylabel("Energy gap")
# plt.yscale("log")
# plt.xscale("log")
plt.legend()
plt.show()

#### field fixed varying hopping - scaling of energy gap

In [138]:
set=[]
lab = []

In [130]:
lab.append(str(Nsites)+" sites, "+str(Nbosons)+" bosons")

###### parameters
min = -5
max = 5
numberofvalues = 100
vals=np.linspace(min,max,numberofvalues)
# vals=np.logspace(-2,2,numberofvalues)

# Different system sizes and number of bosons - choose number of bosons to be equal to the system size

deltas=np.zeros((2,len(vals)))
energy0=np.zeros((2,len(vals)))
energy1=np.zeros((2,len(vals)))
for i in range(len(vals)):
    val = vals[i]
    hopping_strength = -1
    field_strength = val
    Hgaugefixed = util.build_H(hopping_strength, field_strength, L_modes, L_spin, P_sparse, basis, True)
    E,V = eigsh(Hgaugefixed,k=2,which='SA')
    delta=np.abs(E[1]-E[0])
    if val==0:
        print("E[0] ",E[0]," E[1] ",E[1]," delta ",delta)
    deltas[0][i]=val
    deltas[1][i]=delta

    energy0[0][i]=val
    energy0[1][i]=E[0]

    energy1[0][i]=val
    energy1[1][i]=E[1]

set.append(deltas)

In [93]:
xlabelplot = r"$|J/\lambda|$"
# plt.title("Z2LGT Energy gaps")
# for i in range(len(set)):
#     plt.plot(energy0[0],energy0[1],label="E0 "+lab[i])
# for i in range(len(set)):
#     plt.plot(energy1[0],energy1[1],label="E1 "+lab[i])
# plt.xlabel(xlabelplot)
# plt.ylabel("Eigenvalue")
# plt.legend()
# plt.show()

plt.title("Z2LGT Energy gap")
for i in range(len(set)):
    plt.plot(np.abs(set[i][0]),np.abs(set[i][1]),label=lab[i])
    # plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(2)),label="$x^{2}$")
    # plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(1)),label="$x$")
plt.xlabel(xlabelplot)
plt.ylabel("Energy gap")
plt.yscale("log")
plt.xscale("log")
plt.ylim(10**(-3))
plt.xlim(0.05)
plt.legend()
plt.show()

In [132]:
xlabelplot = r"$J/\lambda$"
plt.title("Z2LGT Energy gap")
for i in range(len(set)):
    plt.plot(set[i][0],set[i][1],label=lab[i])
    # plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(2)),label="$x^{2}$")
    # plt.plot(np.abs(set[i][0]),np.abs(set[i][0]**(1)),label="$x$")
plt.xlabel(xlabelplot)
plt.ylabel("Energy gap")
# plt.yscale("log")
# plt.xscale("log")
# plt.ylim(10**(-3))
# plt.xlim(0.05)
plt.legend()
plt.show()

# Adiabatic

### define initial state (ground state of X field)

In [12]:
field = [[-1,i] for i in range(L_spin)]
static=[["x|",field]]#,["|nn",density]]
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
H1 = hamiltonian(static,[],basis=basis,**no_checks)

H = hamiltonian(static, [], basis=basis, **no_checks)
H_sparse = H.tocsr()
Hgaugefixed = P_sparse @ H_sparse @ P_sparse.T.conj()
_,psi = eigsh(Hgaugefixed,k=1, which='SA')

### Time evolve

get real groundstate

In [13]:
hop=[[-1.0,i,i,i+1] for i in range(L_modes-1)]
# DENSITY AT ZERO - density should be converged for 0.01
# density = [[0.01,i,i] for i in range(L_modes)]
field = [[h,i] for i in range(L_spin)]
# static=[["z|+-",hop],["z|-+",hop],["x|",field]]
static=[["z|+-",hop],["z|-+",hop],["x|",field]]#,["|nn",density]]
###### setting up operators
# set up hamiltonian dictionary and observable (imbalance I)
no_checks = dict(check_pcon=False, check_symm=False, check_herm=False)
H = hamiltonian(static, [], basis=basis, **no_checks)
H_sparse = H.tocsr()
Hgaugefixed = P_sparse @ H_sparse @ P_sparse.T.conj()
_,psi1 = eigsh(Hgaugefixed, k=1, which='SA')
# psi1=np.dot(P.T.conj(),psi1)[:,0]

In [15]:
##### create model
def drive(t,t_0):
    return t/t_0

res=[]
t0s=np.logspace(-3,2,20)
for t0 in t0s:
    drive_args=[t0]
    hop=[[-1.0,i,i,i+1] for i in range(L_modes-1)]
    density = [[0.01,i,i] for i in range(L_modes)]
    field = [[h,i] for i in range(L_spin)]
    # static=[["z|+-",hop],["z|-+",hop],["x|",field]]
    # set up hamiltonian dictionary and observable (imbalance I)
    no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)
    static=[["x|",field],["|nn",density]]
    dynamic=[]
    Hs = hamiltonian(static,dynamic,basis=basis,**no_checks)
    static = []
    dynamic = [["z|+-",hop,drive,drive_args],["z|-+",hop,drive,drive_args]]
    Hd = hamiltonian(static,dynamic,basis=basis,**no_checks)
    Hs = Hs.tocsr()
    Hd = Hd.tocsr()

    Hsgaugefixed = P_sparse @ Hd @ P_sparse.T.conj()
    Hdgaugefixed = P_sparse @ Hd @ P_sparse.T.conj()

    def adiabatic(time,phi):
        phi_dot = -1j*Hsgaugefixed@phi
        phi_dot += -1j*drive(time,t0)*Hdgaugefixed@phi
        return phi_dot

    # psi_t=H.evolve(psi,0.0,[t0],iterate=False,rtol=1E-9,atol=1E-9)
    psi_t=evolve(psi,0.0,[t0],adiabatic,iterate=False,rtol=1E-9,atol=1E-9)
    # calculate fidelity
    res.append(np.abs(np.dot(psi_t[:,0,0].conj(),psi1))**2)
plt.plot(t0s,res,'.')
plt.xscale('log')
plt.show()